In [6]:
import random
from sympy.combinatorics import Permutation, PermutationGroup

def subgroups_of_Sm(N: int) -> dict:
    # вычисление параметров
    m = 4 + (N % 5)
    mod5 = N % 5
    
    if mod5 == 0:
        p, s, r, t = 29, 5, 59, 9
    elif mod5 == 1:
        p, s, r, t = 31, 4, 60, 8
    elif mod5 == 2:
        p, s, r, t = 37, 3, 38, 7
    elif mod5 == 3:
        p, s, r, t = 23, 17, 45, 12
    else:
        p, s, r, t = 19, 15, 44, 14

    # создаем симметрическую группу S_m
    S_m = PermutationGroup(Permutation(1, 2), Permutation(1, 2, 3, 4, 5, 6, 7))
    
    # получаем подгруппы
    all_subgroups = []
    try:
        all_subgroups = S_m.subgroups
    except:
        # альтернативный подход если прямой метод не работает
        trivial = PermutationGroup(Permutation(m))
        all_subgroups.append(trivial)
        
        for i in range(1, m):
            cycle = Permutation(*[(j % m) + 1 for j in range(i, i + m)])
            cyclic_subgroup = PermutationGroup(cycle)
            if cyclic_subgroup not in all_subgroups:
                all_subgroups.append(cyclic_subgroup)
        
        for i in range(1, m):
            for j in range(i + 1, m + 1):
                transposition = Permutation(i, j)
                trans_subgroup = PermutationGroup(transposition)
                if trans_subgroup not in all_subgroups:
                    all_subgroups.append(trans_subgroup)
        
        A_m = S_m.derived_subgroup()
        if A_m not in all_subgroups:
            all_subgroups.append(A_m)
        
        all_subgroups.append(S_m)
    
    # убираем дубликаты и сортируем
    unique_subgroups = []
    for subgroup in all_subgroups:
        if subgroup not in unique_subgroups:
            unique_subgroups.append(subgroup)
    unique_subgroups.sort(key=lambda x: x.order())
    
    # случайная подгруппа
    random_subgroup = random.choice(unique_subgroups)
    
    # выбранная подгруппа для анализа смежных классов
    subgroup_index = N % len(unique_subgroups)
    selected_subgroup = unique_subgroups[subgroup_index]
    
    # строим смежные классы
    group_elements = list(S_m.generate())
    
    left_cosets_dict = {}
    for g in group_elements:
        coset = set(g * h for h in selected_subgroup.elements)
        if not any(coset == existing_coset for existing_coset in left_cosets_dict.values()):
            left_cosets_dict[g] = coset
    
    right_cosets_dict = {}
    for g in group_elements:
        coset = set(h * g for h in selected_subgroup.elements)
        if not any(coset == existing_coset for existing_coset in right_cosets_dict.values()):
            right_cosets_dict[g] = coset
    
    left_cosets = list(left_cosets_dict.values())
    right_cosets = list(right_cosets_dict.values())
    
    index = len(left_cosets)
    is_normal = (left_cosets == right_cosets)
    
    return {
        'total_subgroups': len(unique_subgroups),
        'random_subgroup': {
            'order': random_subgroup.order(),
            'elements': [str(perm) for perm in random_subgroup.elements]
        },
        'selected_subgroup': {
            'index_in_list': subgroup_index,
            'order': selected_subgroup.order(),
            'elements': [str(perm) for perm in selected_subgroup.elements]
        },
        'coset_analysis': {
            'index': index,
            'is_normal': is_normal,
            'left_cosets_count': len(left_cosets),
            'right_cosets_count': len(right_cosets)
        }
    }

In [7]:
def element_powers_in_Sm(N: int) -> dict:
    '''
    порядки элементов g^n1, g^n2, g^n3 и порядки циклических подгрупп
    '''
    m = 4 + (N % 5)
    n1 = N % 6
    n2 = (N + 1) % 6
    n3 = (N + 2) % 6
    
    S_m = PermutationGroup(Permutation(1, 2), Permutation(1, 2, 3, 4, 5, 6, 7))
    elements = list(S_m.generate())
    
    g = elements[N % len(elements)]
    
    def element_order(perm):
        return perm.order()
    
    g_n1 = g ** n1 if n1 > 0 else S_m.identity
    g_n2 = g ** n2 if n2 > 0 else S_m.identity
    g_n3 = g ** n3 if n3 > 0 else S_m.identity
    
    return {
        'g': str(g),
        'g_order': element_order(g),
        'g_n1': {
            'element': str(g_n1),
            'order': element_order(g_n1),
            'subgroup_order': element_order(g_n1)
        },
        'g_n2': {
            'element': str(g_n2),
            'order': element_order(g_n2),
            'subgroup_order': element_order(g_n2)
        },
        'g_n3': {
            'element': str(g_n3),
            'order': element_order(g_n3),
            'subgroup_order': element_order(g_n3)
        }
    }


def solve_sigma_power_eq(N: int) -> dict:
    '''
    все решения уравнения sigma^n = (1 2 3 ... m-1)
    '''
    m = 4 + (N % 5)
    n = 2 + (N % 10)
    
    target_cycle = list(range(1, m))
    target = Permutation(*target_cycle)
    
    S_m = PermutationGroup(Permutation(1, 2), Permutation(1, 2, 3, 4, 5, 6, 7))
    
    solutions = []
    for perm in S_m.generate():
        if perm ** n == target:
            solutions.append(perm)
    
    random_solutions = random.sample(solutions, min(3, len(solutions))) if solutions else []
    
    return {
        'equation': f'σ^{n} = {target}',
        'total_solutions': len(solutions),
        'random_solutions': [str(sol) for sol in random_solutions],
    }


def elements_of_order_k_in_cyclic_group(N: int) -> dict:
    '''
    элементы g такие что g^k = e и элементы порядка k в циклической группе
    '''
    m = 4 + (N % 5)
    k = 1 + (N % 7)
    
    elements_gk_e = []
    elements_order_k = []
    
    for i in range(m):
        if (i * k) % m == 0:
            elements_gk_e.append(i)
        if gcd(i, m) == 1 and pow(i, k, m) == 1:
            if all(pow(i, d, m) != 1 for d in range(1, k) if k % d == 0):
                elements_order_k.append(i)
    
    return {
        'group_order': m,
        'k': k,
        'elements_gk_e': elements_gk_e,
        'elements_order_k': elements_order_k
    }


def subgroups_of_Zm_star(N: int) -> list:
    '''
    подгруппы мультипликативной группы Zm*
    '''
    m = 4 + (N % 5)
    
    def get_Zm_star(m):
        return [i for i in range(1, m) if gcd(i, m) == 1]
    
    def is_subgroup(elements, group):
        if not elements:
            return False
        for a in elements:
            for b in elements:
                if (a * b) % m not in elements:
                    return False
            if pow(a, -1, m) not in elements:
                return False
        return True
    
    Zm_star = get_Zm_star(m)
    all_subsets = []
    
    for r in range(1, len(Zm_star) + 1):
        for subset in combinations(Zm_star, r):
            if is_subgroup(subset, Zm_star):
                all_subsets.append(sorted(subset))
    
    return sorted(all_subsets, key=len)


def order_of_sr(N: int) -> int:
    '''
    порядок элемента s^r в Zp*
    '''
    mod5 = N % 5
    if mod5 == 0:
        p, s, r = 29, 5, 59
    elif mod5 == 1:
        p, s, r = 31, 4, 60
    elif mod5 == 2:
        p, s, r = 37, 3, 38
    elif mod5 == 3:
        p, s, r = 23, 17, 45
    else:
        p, s, r = 19, 15, 44
    
    s_r = pow(s, r, p)
    order = 1
    current = s_r
    while current != 1:
        current = (current * s_r) % p
        order += 1
    
    return order


def order_and_primitivity_of_t(N: int) -> dict:
    '''
    порядок элемента t + проверить, является ли он образующим
    '''
    #элемент является примитивным корнем, если его порядок равен p-1
    mod5 = N % 5
    if mod5 == 0:
        p, t = 29, 9
    elif mod5 == 1:
        p, t = 31, 8
    elif mod5 == 2:
        p, t = 37, 7
    elif mod5 == 3:
        p, t = 23, 12
    else:
        p, t = 19, 14
    
    order = 1
    current = t
    while current != 1:
        current = (current * t) % p
        order += 1
    
    is_primitive = order == p - 1
    
    return {
        't': t,
        'p': p,
        'order': order,
        'is_primitive': is_primitive
    }


def generators_of_Zm_star(N: int) -> list:
    '''
    примитивные корни циклической группы Zm*
    '''
    m = 4 + (N % 5)
    
    def is_primitive_root(a, m):
        if gcd(a, m) != 1:
            return False
        order = 1
        current = a
        while current != 1:
            current = (current * a) % m
            order += 1
        return order == len([i for i in range(1, m) if gcd(i, m) == 1])
    
    generators = []
    for a in range(1, m):
        if gcd(a, m) == 1 and is_primitive_root(a, m):
            generators.append(a)
    
    return generators


def cyclic_subgroup_in_Zm_additive(N: int) -> dict:
    ''' 
    нахождение циклич подгруппы, порождённой элементом t mod m
    '''
    #подгруппа состоит из всех кратных t по модулю m, генераторы - элементы, взаимно простые с порядком подгруппы
    m = 4 + (N % 5)
    mod5 = N % 5
    if mod5 == 0:
        t = 9
    elif mod5 == 1:
        t = 8
    elif mod5 == 2:
        t = 7
    elif mod5 == 3:
        t = 12
    else:
        t = 14
    
    t_mod = t % m
    subgroup = set()
    current = 0
    for i in range(m):
        current = (current + t_mod) % m
        subgroup.add(current)
        if current == 0:
            break
    
    subgroup = sorted(subgroup)
    order = len(subgroup)
    
    generators = []
    for elem in subgroup:
        if elem != 0:
            sub_subgroup = set()
            current_sub = 0
            for i in range(m):
                current_sub = (current_sub + elem) % m
                sub_subgroup.add(current_sub)
                if current_sub == 0:
                    break
            if len(sub_subgroup) == order:
                generators.append(elem)
    
    return {
        't': t,
        't_mod_m': t_mod,
        'subgroup': subgroup,
        'order': order,
        'generators': generators
    }


def isomorphism_of_cyclic_subgroup_Zm_star(N: int) -> dict:
    '''
    циклическая подгруппа в Zm* и её изоморфизм с подгруппой в Sd
    '''
    m = 4 + (N % 5)
    mod5 = N % 5
    if mod5 == 0:
        t = 9
    elif mod5 == 1:
        t = 8
    elif mod5 == 2:
        t = 7
    elif mod5 == 3:
        t = 12
    else:
        t = 14
    
    t_mod = t % m
    
    subgroup = set()
    current = 1
    for i in range(m):
        current = (current * t_mod) % m
        subgroup.add(current)
        if current == 1:
            break
    
    subgroup = sorted(subgroup)
    d = len(subgroup)
    
    S_d = PermutationGroup(Permutation(1, 2), Permutation(1, 2, 3, 4, 5, 6, 7))
    cyclic_subgroup_Sd = None
    
    for perm in S_d.generate():
        if perm.order() == d:
            cyclic_subgroup = PermutationGroup(perm)
            if cyclic_subgroup.order() == d:
                cyclic_subgroup_Sd = cyclic_subgroup
                break
    
    return {
        'subgroup_Zm*': subgroup,
        'order': d,
        'isomorphic_Sd_subgroup': str(cyclic_subgroup_Sd) if cyclic_subgroup_Sd else "Не найдена",
        'generator_Sd': str(cyclic_subgroup_Sd.generators[0]) if cyclic_subgroup_Sd else None
    }